<a href="https://colab.research.google.com/github/nntu079/Basic-Pytorch/blob/main/Basic_Neural_Network_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
# Create fully connected network
class NN(nn.Module):
  def __init__(self,input_size,num_class):
    super(NN,self).__init__()
    self.fc1=nn.Linear(input_size,50)
    self.fc2=nn.Linear(50,num_class)
  def forward(self,x):
    x=F.relu(self.fc1(x));
    x=self.fc2(x);
    return x;

model = NN(784,10);
x=torch.randn(64,784);

print(x)
print(model(x).shape)

tensor([[-2.5258,  2.0331,  1.8623,  ...,  0.4109,  0.5433,  0.9034],
        [-0.0344, -0.7717,  0.8710,  ...,  1.2406, -1.4439, -2.4764],
        [-0.9453,  0.7711,  0.6949,  ...,  2.0984,  2.1727,  0.8435],
        ...,
        [-0.0330, -0.2642, -0.2505,  ..., -0.3868,  1.3435, -1.1677],
        [ 2.2798,  2.1562, -0.1514,  ..., -0.5325, -0.4321, -1.8356],
        [ 0.4693, -1.0760, -0.7208,  ...,  0.1718,  1.5647,  0.0990]])
torch.Size([64, 10])


In [3]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyperparameters
input_size=784
num_class=10
learning_rate=0.001
batch_size=64
num_epochs=1

In [5]:
# Load Data
patch=r'/content/drive/MyDrive/Môn học/Pytorch/mnist';
train_dataset=datasets.MNIST(root=patch,train=True,transform=transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root=patch,train=False,transform=transforms.ToTensor(),download=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [6]:
# Initialize Network
model= NN(input_size,num_class).to(device)

In [7]:
# Loss and optimizer
criterion= nn.CrossEntropyLoss();
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
# Train Network
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    # Get data to Cuda if possible
    data=data.to(device=device);
    targets=targets.to(device=device);
    
    # Get data ro correct shape
    data=data.reshape(data.shape[0],-1)

    # Forward
    scores=model(data);
    loss= criterion(scores,targets);

    # Backward
    optimizer.zero_grad();
    loss.backward();

    # Gradient Descent
    optimizer.step();



In [10]:
# Check accuracy on training and test to see how good our model

def check_accuracy(loader,model):
  num_correct=0;
  num_samples=0;

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      x=x.reshape(x.shape[0],-1)

      scores=model(x);
      _,predictions=scores.max(1);

      num_correct += (predictions==y).sum();
      num_samples += predictions.size(0);

    print(num_correct/num_samples,num_correct,num_samples)

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)



tensor(0.9340) tensor(56039) 60000
tensor(0.9344) tensor(9344) 10000


tensor([[-4.9927e+00, -2.2710e+00,  1.2068e+00,  4.2140e+00, -1.1682e+00,
         -2.3545e+00, -1.0894e+01,  3.4313e+00, -3.1012e+00, -4.0646e+00],
        [-2.2412e-01,  3.7218e-01, -4.1703e+00, -9.6101e-01, -3.8675e+00,
          2.2611e+00, -1.1054e+00,  2.2691e+00, -4.8716e+00, -1.1171e+00],
        [-1.0813e+00, -3.1835e+00,  4.5987e+00, -1.0554e+00, -3.4049e-01,
         -7.7984e-01, -1.7068e+00, -1.3089e+00, -1.0751e+00,  3.1857e-01],
        [-3.2975e+00, -5.0034e-02,  8.3567e-01, -2.9886e+00,  3.0743e-01,
         -5.3067e-01, -7.2478e-01, -8.1968e-01, -3.0712e+00, -1.3130e+00],
        [-1.7488e+00, -1.0201e+00, -7.5978e-01, -2.1148e+00,  6.5367e-01,
          3.3602e+00, -1.0666e+00, -5.1667e+00,  1.2563e+00,  2.9880e+00],
        [ 1.3747e+00, -3.3420e+00, -2.4531e+00, -1.0888e+00, -9.8054e-01,
          1.3227e-01, -1.6449e+00,  1.1165e+00, -4.6269e-01,  1.3724e+00],
        [-3.8354e+00, -1.2057e+00,  1.0796e+00, -1.7737e-01, -2.3491e+00,
         -2.7994e+00,  1.5042e+0